In [99]:
import pandas as pd
import numpy as np
import os
# import subprocess
# subprocess.check_call(['pip', 'install', 'numpy']) # install pkg
# subprocess.check_call(['pip', 'install',"--upgrade", 'numpy']) # upgrade pkg

In [100]:
path = 'QTRWeaks of Month/'

files = []
filesNoAdd = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.csv' in file:
            files.append(os.path.join(r, file))
            filesNoAdd.append(file)

for f in filesNoAdd:
    print(f)


08-10-2019.csv
08-17-2019.csv
08-24-2019.csv
08-31-2019.csv


In [101]:
numListOfBranch = ['301 Co','302 Co','306 Co','307 Co']
listOfBranch = ['301 Co', '301.Arq COST', '301.Cost Price', '301.V.S.P.', '301.AVG. WEEK',
                '302 Co', '302.Arq COST', '302.Cost Price', '302.V.S.P.', '302.AVG. WEEK',
                '306 Co', '306.Arq COST', '306.Cost Price', '306.V.S.P.', '306.AVG. WEEK',
                '307 Co', '307.Arq COST', '307.Cost Price', '307.V.S.P.', '307.AVG. WEEK',
                 'Latest SOH']

productDetail = ['Sku', 'UPC', 'Catalogue N', 'Title', 'Label' ,'Arq COST', 'Cost Price' ,'V.S.P.']
header = productDetail.copy()
header.extend(listOfBranch.copy())

In [102]:
def findID(sku, Data):
    numb = 0
    for SKUref in Data['Sku']:
        
        if str(sku) == str(SKUref):
            return numb
        numb += 1
    return "This good with Sku of " + sku + " is not in data." 

In [103]:
def buildList(num, ide, productDetail, listOfBranch, rawData, final):
    producList = []
    qSum = 0
    for det in productDetail:
        producList.append(rawData[det][ide])
    for branch in listOfBranch:
        branch = branch.replace(' Co','')
        qbranch = str(branch + '.Quantit')
        head = str(branch + '.Sales Quantity')
        if head in rawData.head(0):
            try:
                producList.append(float(rawData[head][ide]))
                producList.append(np.round(float(rawData[head][ide]) * float(rawData['Arq COST'][ide]),2))
                producList.append(np.round(float(rawData[head][ide]) * float(rawData['Cost Price'][ide]),2))
                producList.append(np.round(float(rawData[head][ide]) * float(rawData['V.S.P.'][ide]),2))
                producList.append(np.round(float(rawData[head][ide]) / float(num),2))
            except:
                print("A number in raw data is string" , head, ide)
        if qbranch in rawData.head(0) and str(branch)!='344'and str(branch)!='390'and final==True:
            try:
                qSum = qSum + rawData[qbranch][ide]
            except:
                print("A number in raw data is string", qbranch, ide)
    producList.append(qSum)

                
    return producList

In [104]:
def listAddition(Dataide, ide, num, productDetail, listOfBranch, rawData, monthRawData, final):
    colid = 8
    qSum = 0
    buf = 0
    for branch in listOfBranch:        
        branch = branch.replace(' Co','')
        qbranch = str(branch + '.Quantit')        
        head = str(branch + '.Sales Quantity')
        if head in rawData.head(0):
            buf = colid + 5
#             print(colid)
            monthRawData.iloc[Dataide, colid] = monthRawData.iloc[Dataide, colid] + rawData[head][ide]
            monthRawData.iloc[Dataide, colid+1] = np.round(monthRawData.iloc[Dataide, colid] * float(rawData['Arq COST'][ide]),2)
            monthRawData.iloc[Dataide, colid+2] = np.round(monthRawData.iloc[Dataide, colid] * float(rawData['Cost Price'][ide]),2)
            monthRawData.iloc[Dataide, colid+3] = np.round(monthRawData.iloc[Dataide, colid] * float(rawData['V.S.P.'][ide]),2)
            monthRawData.iloc[Dataide, colid+4] = np.round(monthRawData.iloc[Dataide, colid] / float(num),2)
#         print(monthRawData)
        colid += 1
        if qbranch in rawData.head(0) and str(branch)!='344'and str(branch)!='390'and final==True:
            try:
                qSum = qSum + rawData[qbranch][ide]
            except:
                print("A number in raw data is string", qbranch, ide)

    if final:
#         print(qSum)
        
        monthRawData.iloc[Dataide,buf] = qSum
        
    return monthRawData

In [105]:
def noStock(sku,rawData):
    sku = str(sku)
    try:
        i = findID(sku, rawData)
        i = int(i)
    except:
        return True
    for branch in numListOfBranch:        
        branch = branch.replace(' Co','')
        qbranch = str(branch + '.Quantit')
        if rawData.loc[i,qbranch] > 0 and str(branch)!='344'and str(branch)!='390':
            return False
    return True


In [106]:
# ArghoobCost
ArghoobCost = pd.read_excel('hand made rteports\ARQOOB Cost.xlsx', encoding = 'latine1')
for f in files:
    rawData = pd.read_csv(f , encoding='latin1')
    i = 0
    for sku in rawData['Sku']:
        j = findID(sku, ArghoobCost)
        rawData.loc[i,'Arq COST'] = ArghoobCost.loc[j, 'Arq COST']
        rawData.loc[i,'UPC'] = ArghoobCost.loc[j, 'UPC Code']
        i += 1
    rawData.to_csv(f, index=False)


In [107]:

monthRawData = pd.DataFrame(columns = list(header))

fcount = len(files)
b = []
finalFile =False
for f in files:
#     f1 = (path + f)
    f1 = (f)
    rawData = pd.read_csv(f1 , encoding='latin1')

    if f == files[-1]:
        finalFile = True
    sid =0
    b = 0
    
    for sku in rawData['Sku']:
        if sku not in list(monthRawData['Sku']):
            df = pd.DataFrame([list(buildList(fcount, sid, productDetail,
                            listOfBranch, rawData, finalFile))], columns= list(header))
            monthRawData = monthRawData.append(df)
            
        else:
#             print(f,sid)
            Dataide = findID(sku, monthRawData)
            b = listAddition(Dataide, sid, fcount, productDetail, listOfBranch, rawData,
                             monthRawData, finalFile)
        sid +=1



In [108]:
totalGoodSale = []
count = 0
for sku in monthRawData['Sku']:
    saleOfGood = []
    for numB in numListOfBranch:
        saleOfGood.append(monthRawData.iloc[count][numB])
    totalGoodSale.append(np.sum(saleOfGood))
    count += 1
monthRawData["total sale of Goods"] = totalGoodSale
status = []
for tot in totalGoodSale:

    if tot <= 0:
        status.append('Non moving')
    if (tot > 0) and (tot <=2):
        status.append('Slow moving')
    if tot > 2:
        status.append('Fast moving')
#
rawData = pd.read_csv(files[-1])

for i in range(len(status)):

    try:
        if noStock(monthRawData.iloc[i]['Sku'],rawData) and status[i]=='Non moving':
            status[i] = 'Run out of stock'
    except:
        continue
#
monthRawData["Status"] = status

In [109]:
monthRawData.to_csv("toalOfMonth.csv", index=False)